In [1]:
#Understat Data Scraping:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
# links is an array of the ends of the urls that I want to access
links = ["Ligue_1/2019", 
         "La_liga/2019",
         "EPL/2019",
         "Bundesliga/2019",
         "Serie_A/2019"]
# understat is the beginning of the url I want to access
understat = "http://understat.com/league/"
# for each of the leagues
for link in links:
    # open the html
    html = urllib.urlopen(understat+link)
    # clean up the html
    soup = BeautifulSoup(html.read())
    # find all the "script" tags
    scripts = soup.find_all("script")
    # for each of the script tags found
    for tag in scripts:
        # if the tag contains playerdata
        if "playersData" in tag.get_text():
            # get that data
            data = tag.get_text()
            # take the portion of the data I want (I don't want the first 32 characters or last 4)
            data = data[32:-4]
    # decode the data
    decoded_data = bytes(data, 'utf-8').decode('unicode_escape')
    # create a dataframe from the decoded data
    df = pd.read_json(decoded_data)
    # break up the link to create the components for a name
    league, part, year = link.partition("/")
    # create a csv for this league
    df.to_csv("Desktop/" + league +"_" + year +".csv")

In [1]:
import pandas as pd
#read in the csvs created above
spain = pd.read_csv("La_liga_2019.csv").drop(columns="Unnamed: 0")
england = pd.read_csv("EPL_2019.csv").drop(columns="Unnamed: 0")
france = pd.read_csv("Ligue_1_2019.csv").drop(columns="Unnamed: 0")
germany = pd.read_csv("Bundesliga_2019.csv").drop(columns="Unnamed: 0")
italy = pd.read_csv("Serie_A_2019.csv").drop(columns="Unnamed: 0")
#create a column in each dataframe denoting league
spain["League"] = "La Liga"
england["League"] = "English Premier League"
france["League"] = "Ligue 1"
germany["League"] = "Bundesliga"
italy["League"] = "Serie A"

In [2]:
italy.head()

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,League
0,1209,Ciro Immobile,26,2189,27,20.222042,7,4.612372,101,47,4,0,F S,Lazio,17,11.847795,16.814746,2.586083,Serie A
1,2371,Cristiano Ronaldo,22,1945,21,17.347312,3,4.153005,130,37,2,0,F,Juventus,14,12.018220,17.612762,5.048570,Serie A
2,594,Romelu Lukaku,25,2147,17,13.692168,2,3.396662,72,36,2,0,F S,Inter,13,10.646972,15.922957,3.348168,Serie A
3,1592,João Pedro,25,2250,16,10.899735,2,1.728891,58,23,5,0,F M,Cagliari,13,7.093240,12.869609,5.093203,Serie A
4,1426,Josip Ilicic,21,1501,15,9.952888,5,6.951148,75,50,1,1,F M S,Atalanta,15,9.952888,22.623260,10.017149,Serie A


In [3]:
# Make a dataframe club that contains all 5 league's data
club = pd.concat(objs=[spain, england, france, germany, italy])
#Fix the index so it doesn't restart at 0 when new league
club = club.set_index("player_name")
club = club.reset_index()

In [4]:
club.head()

,player_name,id,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup,League
0,Lionel Messi,2097,22,1886,19,15.177277,12,9.938458,105,54,3,0,F M S,Barcelona,16,12.947444,23.942403,6.779710,La Liga
1,Karim Benzema,2370,26,2231,14,16.017713,6,5.163030,96,44,0,0,F S,Real Madrid,11,13.787881,23.157893,8.101761,La Liga
2,Lucas Pérez,1700,26,1943,11,8.928882,5,5.339243,39,48,3,0,F S,Alaves,6,5.212880,10.012859,2.958536,La Liga
3,Luis Suárez,2098,17,1274,11,9.322706,7,2.758954,52,19,4,0,F S,Barcelona,10,8.579429,13.036603,2.784715,La Liga
4,Gerard Moreno,2120,25,2071,11,11.456654,3,5.074703,70,40,4,0,F M S,Villarreal,11,10.713376,17.806753,5.334714,La Liga


In [5]:
#Need to find errors in the player_name column
club[club["player_name"].str.contains("&#039")]
#Find and replace '&#039;' to a single apostrophe
#temp_club = club.replace(to_replace = "*\&#039;*", value = "jk")
import html
html.unescape("M&#039;Baye Niang")
club["player_name"] = club["player_name"].apply(html.unescape)

In [6]:
#Drop a bunch of unnecessary columns: id(specific to understat website), position, yellow and red cards,
club = club.drop(columns = ["id", "red_cards", "yellow_cards", "xG", "games", "team_title", "npxG","shots", "assists", "goals", "xG", "xA", "key_passes", "position", "League"])

In [34]:
#Standardize columns to be 'per-90 minutes' to see efficiency and allow for players who did not play as many games
# All the statistics that have to be changed to 'per 90 minutes'
stats = ["npg", "xGBuildup", "xGChain"]
#Create a new column for the new stats per 90 mins, and then drop the old stats
for stat in stats:
    new_name = stat + " per 90 mins"
    club[new_name] = club[stat] / club["time"] * 90
club.head()

,player_name,time,npg,xGChain,xGBuildup,npg per 90 mins,xGBuildup per 90 mins,xGChain per 90 mins
0,Lionel Messi,1886,16,23.942403,6.779710,0.763521,0.323528,1.142532
1,Karim Benzema,2231,11,23.157893,8.101761,0.443747,0.326830,0.934205
2,Lucas Pérez,1943,6,10.012859,2.958536,0.277921,0.137040,0.463797
3,Luis Suárez,1274,10,13.036603,2.784715,0.706436,0.196722,0.920953
4,Gerard Moreno,2071,11,17.806753,5.334714,0.478030,0.231832,0.773833


In [7]:
club.head()

,player_name,time,npg,xGChain,xGBuildup
0,Lionel Messi,1886,16,23.942403,6.779710
1,Karim Benzema,2231,11,23.157893,8.101761
2,Lucas Pérez,1943,6,10.012859,2.958536
3,Luis Suárez,1274,10,13.036603,2.784715
4,Gerard Moreno,2071,11,17.806753,5.334714


In [18]:
#creates a CSV with the data on your desktop (mac)
club.to_csv("Desktop/understatData.csv")